## Pyspark

In [0]:
# Loading Dataset
csv_file = "dbfs:/FileStore/shared_uploads/yateed1437@gmail.com/assignment.csv"
df = spark.read.format("csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load(csv_file)

In [0]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- nametype: string (nullable = true)
 |-- recclass: string (nullable = true)
 |-- mass (g): double (nullable = true)
 |-- fall: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- reclat: double (nullable = true)
 |-- reclong: double (nullable = true)
 |-- GeoLocation: string (nullable = true)



In [0]:
col_list = df.columns
print(col_list)

['name', 'id', 'nametype', 'recclass', 'mass (g)', 'fall', 'year', 'reclat', 'reclong', 'GeoLocation']


In [0]:
from pyspark.sql.functions import when, count, col, isnan
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in col_list]).display()

name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,0,0,0,131,0,291,7315,7315,7315


In [0]:
# Group By Clause
spark.sql("""
            SELECT year, count(year) from dataset
            group by year
          """).display()

year,count(year)
1959,16
1829,4
1896,10
1990,1518
1903,22
1884,11
1975,337
1977,421
1766,1
1787,1


## Replace the year values with minimum values count that are before 1500

In [0]:
list_values = [row.year for row in df.select(df.year).filter(df.year < 1500).collect()]
print(list_values)

In [0]:
new_year_value = 1950

# Update values in the 'year' column
df = df.withColumn('year', when(df['year'].isin(list_values), new_year_value).otherwise(df['year']))

In [0]:
# Let us check for the values
df.select(col("*")).filter(df.year < 1500).display()

name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation


In [0]:
#reclat and reclong
from pyspark.sql.functions import count
df.groupBy("reclat").agg(count("reclat").alias("Count")).orderBy("Count", ascending = False).display()

reclat,Count
0.0,6438
-71.5,4761
-84.0,3040
-72.0,1506
-79.68333,1130
-76.71667,680
-76.18333,539
-84.21667,263
-86.36667,226
-86.71667,217
